
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header = True, inferSchema = True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[23]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
## Handling categorical features

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCols= ['sex','smoker','day','time'], outputCols=['sex_indexed',"smoker_indexed","day_indexed","time_indexed"])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['tip','sex_indexed','smoker_indexed','day_indexed','time_indexed'], outputCol="Independent_features")

In [0]:
output = feature_assembler.transform(df_r)

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent_features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,1.0,0.0,1.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|(5,[0,3],[1.66,1.0])|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| (5,[0,3],[3.5,1.0])|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|(5,[0,3],[3.31,1.0])|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,1.0,0.0,1.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
output.select('Independent_features').show()

+--------------------+
|Independent_features|
+--------------------+
|[1.01,1.0,0.0,1.0...|
|(5,[0,3],[1.66,1.0])|
| (5,[0,3],[3.5,1.0])|
|(5,[0,3],[3.31,1.0])|
|[3.61,1.0,0.0,1.0...|
|(5,[0,3],[4.71,1.0])|
| (5,[0,3],[2.0,1.0])|
|(5,[0,3],[3.12,1.0])|
|(5,[0,3],[1.96,1.0])|
|(5,[0,3],[3.23,1.0])|
|(5,[0,3],[1.71,1.0])|
|[5.0,1.0,0.0,1.0,...|
|(5,[0,3],[1.57,1.0])|
| (5,[0,3],[3.0,1.0])|
|[3.02,1.0,0.0,1.0...|
|(5,[0,3],[3.92,1.0])|
|[1.67,1.0,0.0,1.0...|
|(5,[0,3],[3.71,1.0])|
|[3.5,1.0,0.0,1.0,...|
|      (5,[0],[3.35])|
+--------------------+
only showing top 20 rows



In [0]:
finalized_data = output.select("Independent_features","total_bill")

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent_features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|(5,[0,3],[1.66,1.0])|     10.34|
| (5,[0,3],[3.5,1.0])|     21.01|
|(5,[0,3],[3.31,1.0])|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|(5,[0,3],[4.71,1.0])|     25.29|
| (5,[0,3],[2.0,1.0])|      8.77|
|(5,[0,3],[3.12,1.0])|     26.88|
|(5,[0,3],[1.96,1.0])|     15.04|
|(5,[0,3],[3.23,1.0])|     14.78|
|(5,[0,3],[1.71,1.0])|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|(5,[0,3],[1.57,1.0])|     15.42|
| (5,[0,3],[3.0,1.0])|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|(5,[0,3],[3.92,1.0])|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|(5,[0,3],[3.71,1.0])|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|      (5,[0],[3.35])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent_features", labelCol="total_bill")
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[33]: DenseVector([3.9603, -1.4943, 1.498, -0.4093, -1.436])

In [0]:
regressor.intercept

Out[34]: 8.699628720362659

In [0]:
pred_results = regressor.evaluate(test_data)

In [0]:
#Final comparision

pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent_features|total_bill|        prediction|
+--------------------+----------+------------------+
|      (5,[0],[1.45])|      9.55| 14.44204038400104|
|      (5,[0],[1.75])|     17.82|15.630125555788291|
|       (5,[0],[2.0])|     12.69|16.620196532277667|
|       (5,[0],[2.0])|     13.37|16.620196532277667|
|       (5,[0],[2.0])|     16.31|16.620196532277667|
|      (5,[0],[2.01])|     20.23|16.659799371337243|
|      (5,[0],[2.72])|     13.28|19.471600944567072|
|      (5,[0],[3.15])|     20.08|21.174523024128796|
|      (5,[0],[3.39])|     11.61|22.124991161558597|
|      (5,[0],[6.73])|     48.27| 35.35233940745666|
| (5,[0,1],[1.0,1.0])|      7.25|11.165654616209256|
| (5,[0,2],[3.0,1.0])|     38.73|  22.0784582776262|
|(5,[0,2],[3.16,1.0])|     15.81|  22.7121037025794|
|(5,[0,2],[5.16,1.0])|     25.89| 30.63267151449441|
|(5,[0,3],[1.32,1.0])|      9.68| 13.51793849528411|
| (5,[0,3],[1.5,3.0])|     12.46|13.4122596364

In [0]:
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

Out[41]: (0.4599360684044378, 4.647754472992644, 44.574472785552196)